# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [7]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [24]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [25]:
show(boston, allcols=true)

506×14 DataFrame
│ Row │ Crim    │ Zn      │ Indus   │ Chas  │ NOx     │ Rm      │ Age     │
│     │ Float64 │ Float64 │ Float64 │ Int64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ 0.00632 │ 18.0    │ 2.31    │ 0     │ 0.538   │ 6.575   │ 65.2    │
│ 2   │ 0.02731 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 6.421   │ 78.9    │
│ 3   │ 0.02729 │ 0.0     │ 7.07    │ 0     │ 0.469   │ 7.185   │ 61.1    │
│ 4   │ 0.03237 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.998   │ 45.8    │
│ 5   │ 0.06905 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 7.147   │ 54.2    │
│ 6   │ 0.02985 │ 0.0     │ 2.18    │ 0     │ 0.458   │ 6.43    │ 58.7    │
│ 7   │ 0.08829 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.012   │ 66.6    │
│ 8   │ 0.14455 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.172   │ 96.1    │
│ 9   │ 0.21124 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 5.631   │ 100.0   │
│ 10  │ 0.17004 │ 12.5    │ 7.87    │ 0     │ 0.524   │ 6.004   │ 85.9 

In [26]:
# 切分訓練資料及測試資料
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.09164,0.0,10.81,0,0.413,6.065,7.8,5.2873,4,305
2,0.19073,22.0,5.86,0,0.431,6.718,17.5,7.8265,7,330
3,28.6558,0.0,18.1,0,0.597,5.155,100.0,1.5894,24,666
4,0.04301,80.0,1.91,0,0.413,5.663,21.9,10.5857,4,334
5,7.40389,0.0,18.1,0,0.597,5.617,97.9,1.4547,24,666
6,0.07022,0.0,4.05,0,0.51,6.02,47.2,3.5549,5,296
7,4.26131,0.0,18.1,0,0.77,6.112,81.3,2.5091,24,666
8,67.9208,0.0,18.1,0,0.693,5.683,100.0,1.4254,24,666
9,0.34006,0.0,21.89,0,0.624,6.458,98.9,2.1185,4,437


In [27]:
model = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat ), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                Estimate  Std. Error     t value  Pr(>|t|)     Lower 95%     Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)   36.1167     5.87961      6.14271      <1e-8    24.5571       47.6763
Crim          -0.0620508  0.0466448   -1.33028      0.1842   -0.153757      0.0296552
Zn             0.0366032  0.0156971    2.33184      0.0202    0.00574184    0.0674645
Indus         -0.0240382  0.0673682   -0.356817     0.7214   -0.156487      0.108411
Chas           2.44857    0.944517     2.5924       0.0099    0.591601      4.30553
NOx          -18.5599     4.23992

In [31]:
pred = predict(model, test)

101-element Array{Union{Missing, Float64},1}:
 24.37141408221631
 23.82672976695713
 12.61527233140859
 13.724934996077641
 14.077074783775151
 25.896308626236934
 19.015604370050692
 11.409413879359047
 19.626414670049783
 26.792742102481725
 22.85049699277405
 17.972019169531713
 18.293788437124714
  ⋮
 25.25856338937318
 16.098157577164738
 19.328728623334378
 18.574571113666003
 24.60661068183594
 28.734457794961088
 15.447693500791125
 17.588302021695505
 12.629483650971085
 33.54922965731473
 23.915599975904556
 22.257616187841464

In [29]:
# 模型評估
GLM.r²(model)

0.7261171269048176

In [30]:
GLM.adjr²(model)

0.7170110467251823